In [1]:
# Add mooncake to sys

import sys

mooncake_path = '/Users/ramonamezquita/Projects/mooncake'
sys.path.append(mooncake_path)

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from mooncake.helper import column_selector
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [5]:
def select_top_n_groups(X, n, group_ids, wrt, aggfunc=sum):
    top_group_ids =  X.groupby(
        group_ids).agg({wrt: aggfunc}).nlargest(n, wrt).index.tolist()
    
    return X.set_index(group_ids).loc[top_group_ids].reset_index()

In [6]:
# Data and description

X = pd.read_csv('sales_example/sales_forecasting_dataset.csv')
date = 'Date'
group_ids = ['Store', 'Dept']
target = 'Weekly_Sales'

# Send group ids to object dtype
for g in group_ids:
    X[g] = X[g].astype(str)
    
# Select subset
n_groups_to_use = 1
X = select_top_n_groups(X, n_groups_to_use, group_ids, target)
X.sort_values(group_ids + [date], inplace=True)

In [8]:
num_selector = column_selector(dtype_include=(int, float), pattern_exclude='time_idx')
transformers = [
    ('num', MinMaxScaler(), num_selector),
    ('cat', OneHotEncoder(), ['IsHoliday'])
]